In [21]:
import json 
with open("/home/vulamanh/Documents/nanographrag/training_embedding/zalo_2022_e2e/zac2022_train.json", "r") as f:
    train_data_1 = json.load(f)

In [ ]:
# prepocess dataset 
preprocess_train_data  = []
for sample in train_data_1["data"]:
    preprocess_train_data.append({"id" : sample["id"],"question" : sample["question"], "context" : sample["text"], "label" : sample["is_long_answer"]})

In [32]:
train_data_1

{'_version_': '1.0',
 '_published_date_': '2022-11-01',
 '_count_': 20857,
 'data': [{'id': '718d41cd997b2b44b0685ac54aa55bd8',
   'question': 'Thủ tướng Trung Quốc là gì',
   'title': 'Trung Quốc',
   'text': 'Thủ tướng Trung Quốc là nhân vật lãnh đạo chính phủ , chủ trì Quốc vụ viện gồm bốn phó thủ tướng cùng người đứng đầu các bộ và uỷ ban cấp bộ . Chủ tịch nước đương nhiệm là Tập Cận Bình , ông cũng là Tổng Bí thư của Đảng Cộng sản Trung Quốc và Chủ tịch Quân uỷ Trung Quốc , do vậy ông là lãnh đạo tối cao của Trung Quốc .',
   'category': 'PARTIAL_ANNOTATION',
   'is_long_answer': True},
  {'id': 'c926e7b0717202618a10dd907d4b4c39',
   'question': 'Đất nước nào không có quân đội',
   'title': '',
   'text': 'có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...',
   'short_candidate_start': 53,
   'short_candidate': 'Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...',
   'answer': 'wiki/

In [31]:
preprocess_train_data

[{'question': 'Thủ tướng Trung Quốc là gì',
  'context': 'Thủ tướng Trung Quốc là nhân vật lãnh đạo chính phủ , chủ trì Quốc vụ viện gồm bốn phó thủ tướng cùng người đứng đầu các bộ và uỷ ban cấp bộ . Chủ tịch nước đương nhiệm là Tập Cận Bình , ông cũng là Tổng Bí thư của Đảng Cộng sản Trung Quốc và Chủ tịch Quân uỷ Trung Quốc , do vậy ông là lãnh đạo tối cao của Trung Quốc .',
  'label': True},
 {'question': 'Đất nước nào không có quân đội',
  'context': 'có 23 quốc gia không có lực lượng quân đội, bao gồm: Costa Rica, Iceland, Panama, Micronesia, Quần đảo Marshall, và Vatican...',
  'label': True},
 {'question': 'Pháp tấn công xâm lược Việt Nam vào ngày tháng năm nào',
  'context': 'Raymondienne (hay Raymonde Dien) sinh ngày 13 tháng 5 năm 1929 tại Pháp là một người phụ nữ (đảng viên Đảng Cộng sản Pháp) đã tham gia phong trào đấu tranh chống Pháp tái xâm lược Việt Nam.',
  'label': False},
 {'question': 'Cờ vua còn có tên gọi nào khác',
  'context': 'Cúp cờ vua thế giới là tên gọi mộ

In [28]:
train_data_2

[{'id': 'u7-1570446247_1',
  'question': 'Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi',
  'title': 'Nguyễn Quang Hải (sinh 1997)',
  'text': 'Năm 2013 , Nguyễn Quang Hải giành chức vô địch U21 quốc gia 2013 cùng với đội trẻ Hà Nội T&T và tạo nên cú sốc khi trở thành cầu thủ 16 tuổi đầu tiên giành được danh hiệu vô địch U21 quốc gia .',
  'label': True},
 {'id': 'u7-1570446247_2',
  'question': 'Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi',
  'title': 'Nguyễn Quang Hải (sinh 1997)',
  'text': 'Sau chức vô địch U-21 quốc gia 2013 , Nguyễn Quang Hải mới 16 tuổi lập tức được HLV Phan Thanh Hùng điền vào danh sách của đội bóng thủ đô tham dự V-League 2014 .',
  'label': True},
 {'id': 'u7-1570446247_0',
  'question': 'Quang Hải giành được chức vô địch U21 quốc gia năm bao nhiêu tuổi',
  'title': 'Nguyễn Quang Hải (sinh 1997)',
  'text': 'Anh bắt đầu gia nhập lò đào tạo trẻ Hà Nội T&T khi mới 9 tuổi vào năm 2006 .',
  'label': False},
 {'id': 'u7-1

In [24]:
import json 
with open("/home/vulamanh/Documents/nanographrag/training_embedding/zalo_wiki_qa/train.json", "r") as f:
    train_data_2 = json.load(f)

In [18]:
len(set(train_data_1 + train_data_2))

TypeError: unhashable type: 'dict'

In [14]:
test_data

[{'__id__': 'test_0001',
  'question': 'Toà nhà nào cao nhất Việt Nam',
  'title': 'The Landmark 81',
  'paragraphs': [{'id': 'p1',
    'text': 'The Landmark 81 là một toà nhà chọc trời trong tổ hợp dự án Vinhomes Tân Cảng , một dự án có tổng mức đầu tư 40.000 tỷ đồng , do Công ty Cổ phần Đầu tư xây dựng Tân Liên Phát thuộc Vingroup làm chủ đầu tư . Toà tháp cao 81 tầng , hiện tại là toà nhà cao nhất Việt Nam và là toà nhà cao nhất Đông Nam Á từ tháng 3 năm 2018 .'},
   {'id': 'p2',
    'text': 'Toà tháp cao 81 tầng , hiện tại là toà nhà cao nhất Việt Nam và là toà nhà cao nhất Đông Nam Á từ tháng 3 năm 2018 . Dự án được xây dựng ở Tân Cảng , quận Bình Thạnh , ven sông Sài Gòn . Dự án được khởi công ngày 26/07/2014 .'}]},
 {'__id__': 'test_0002',
  'question': 'Messi thuận chân nào',
  'title': 'Lionel Messi',
  'paragraphs': [{'id': 'p1',
    'text': 'Đôi chân ngắn nhưng mạnh mẽ giúp Messi dễ dàng tăng tốc và sải chân nhanh giúp anh kiểm soát bóng tốt khi rê dắt ở tốc độ cao . Cựu huấ